This script evaluates the performance of different graph-aware architectures in a node classification problem. Several datasets are employed paying special attention to the homophily ratio.

In [1]:
import time
import numpy as np
from pandas import DataFrame
import matplotlib.pyplot as plt
import dgl
import networkx as nx
import torch
import torch.nn as nn


import utils
from gsp_utils.baselines_archs import GCNN_2L
from gsp_utils.baselines_models import NodeClassModel, GF_NodeClassModel
from gsp_utils.data import normalize_gso
from src.arch import GFGCN, GFGCNLayer, GFGCN_noh_Layer, GFGCN_Spows

# SEED = 0
SEED = 15
PATH = 'results/diff_filters/'
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
print(device)

torch.manual_seed(SEED)

cuda:1


In [2]:
def print_full_results(accs, ellapsed_time, datasets, exps):
    mean_accs = accs.mean(axis=2)
    med_accs = np.median(accs, axis=2)
    std_accs = accs.std(axis=2)
    mean_t = ellapsed_time.mean(axis=2)

    for i, dataset_name in enumerate(datasets):
        graph = getattr(dgl.data, dataset_name)(verbose=False)[0]
        edge_hom = dgl.edge_homophily(graph, graph.ndata['label'])

        print(f'{dataset_name} (homophily ratio: {edge_hom:.3f})')
        for j, exp in enumerate(exps):
            print(f'\t- {exp["leg"]}:\tmean: {mean_accs[j,i]:.3f} - std: {std_accs[j,i]:.4f} - med: {med_accs[j,i]:.3f} - time: {mean_t[j,i]:.2f} mins')
        
        print()

def summary_table(accs, datasets, exps, median=False):
    mean_accs = accs.mean(axis=2)
    cols_name = []
    for dataset_name in datasets:
        graph = getattr(dgl.data, dataset_name)(verbose=False)[0]
        edge_hom = dgl.edge_homophily(graph, graph.ndata['label'])
        cols_name.append(f'{dataset_name} ({edge_hom:.2f})')

    index_name = [exp['leg'] for exp in exps]

    return DataFrame(mean_accs, columns=cols_name, index=index_name)


In [3]:
DATASETS = ['TexasDataset',  'WisconsinDataset', 'CornellDataset', 'CiteseerGraphDataset', 'CoraGraphDataset']

EXPS = [{'model': 'Kipf', 'norm': 'both', 'leg': 'Kipf-normA'},
        {'model': 'Kipf', 'norm': 'none', 'leg': 'Kipf-A'},
        {'model': 'GFGCN', 'L': 2, 'K': 2, 'h0': .01, 'norm': True, 'leg': 'GCN-normA'},
        {'model': 'GFGCN', 'L': 2, 'K': 2, 'h0': .01, 'norm': False, 'leg': 'GCN-A'},
        {'model': 'GFGCN', 'L': 2, 'K': 2, 'h0': 1, 'norm': True, 'leg': 'GCN-normA-1'},
        {'model': 'GFGCN', 'L': 2, 'K': 2, 'h0': 1, 'norm': False, 'leg': 'GCN-A-1'},
        {'model': 'GFGCN', 'L': 4, 'K': 2, 'h0': .01, 'norm': False, 'leg': 'GCN-A-4-2'},
        {'model': 'H-GFGCN', 'L': 2, 'K': 2, 'norm': True, 'leg': 'GCN-normH'},
        {'model': 'H-GFGCN', 'L': 2, 'K': 2, 'norm': False, 'leg': 'GCN-H'},
        {'model': 'noh-GFGCN', 'L': 2, 'K': 2, 'norm': True, 'leg': 'W-GCN-normA'},
        {'model': 'noh-GFGCN', 'L': 2, 'K': 2, 'norm': False, 'leg': 'W-GCN-A'},]

## Original Parameters

In [4]:
# Common parameters - training
N_RUNS = 25
N_EPOCHS = 200
LR = .01
WD = 5e-4

# Common parameters - GNN
N_LAYERS = 2
K = 2
# IN_DIM = feat.shape[1]
# OUT_DIM = n_class
HID_DIM = 16
DROPOUT = 0
ACT = nn.ReLU()
LAST_ACT = nn.LogSoftmax(dim=1)
LOSS_FN = nn.NLLLoss()

In [5]:
best_accs = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
accs_best_val = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
ellapsed_times = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
for j, dataset in enumerate(DATASETS):
    A, feat, labels, n_class, masks = utils.get_data_dgl(dataset, dev=device)
    N = A.shape[0]
    in_dim = feat.shape[1]
    out_dim = n_class

    print(dataset)

    for i in range(N_RUNS):
        print(f'{i}:', end=' ')
        for k, exp in enumerate(EXPS):
            t_i = time.time()
            if exp['model'] == 'Kipf':
                arch = GCNN_2L(in_dim, HID_DIM, out_dim, act=ACT, last_act=LAST_ACT,
                               dropout=DROPOUT, norm=exp['norm'])
                S = dgl.from_networkx(nx.from_numpy_array(A)).add_self_loop().to(device)

            elif exp['model'] == 'GFGCN':
                arch = GFGCN(in_dim, HID_DIM, out_dim, exp['L'], exp['K'], act=ACT, last_act=LAST_ACT,
                         dropout=DROPOUT, diff_layer=GFGCNLayer, init_h0=exp['h0'])
                if exp['norm']:
                    S = torch.Tensor(normalize_gso(A, 'both')).to(device)
                else:
                    S = torch.Tensor(A).to(device)

            elif exp['model'] == 'H-GFGCN':
                arch = GFGCN_Spows(in_dim, HID_DIM, out_dim, exp['L'], exp['K'], act=ACT, last_act=LAST_ACT,
                                   dropout=DROPOUT, norm=exp['norm'], dev=device)
                S = torch.Tensor(A).to(device)

            elif exp['model'] == 'noh-GFGCN':
                arch = GFGCN(in_dim, HID_DIM, out_dim, exp['L'], exp['K'], act=ACT, last_act=LAST_ACT,
                             dropout=DROPOUT, diff_layer=GFGCN_noh_Layer)
                if exp['norm']:
                    S = torch.Tensor(normalize_gso(A, 'both')).to(device)
                else:
                    S = torch.Tensor(A).to(device)
            else:
                raise Exception(f'ERROR: unknown architecture {exp["model"]}')

            if exp['model'] in ['Kipf', 'noh-GFGCN']:
                model = NodeClassModel(arch, S, masks, LOSS_FN, device=device)
            else:
                model = GF_NodeClassModel(arch, S, exp['K'], masks, LOSS_FN, device=device)

            loss, acc = model.train(feat, labels, N_EPOCHS, LR, WD)
            ellapsed_t = (time.time()-t_i)/60
            best_accs[k,j,i] = np.max(acc["test"])
            accs_best_val[k,j,i] = acc["test"][np.argmax(acc["val"])]

            ellapsed_times[k,j,i] = ellapsed_t
            print(f'{accs_best_val[k,j,i]:.3f} ({best_accs[k,j,i]:.3f})', end=' - ')
            
        print()      
    print()


print_full_results(accs_best_val, ellapsed_times, DATASETS, EXPS)
table_acc1 = summary_table(best_accs, DATASETS, EXPS)
table_acc_val1 = summary_table(accs_best_val, DATASETS, EXPS)

TexasDataset
0: 

/home/srey/Investigacion/robust_minmax_gnn/gsp_utils/data.py:43: RuntimeWarning: divide by zero encountered in divide
  D_inv = np.diag(1/S.sum(1))
/home/srey/Investigacion/robust_minmax_gnn/gsp_utils/data.py:51: RuntimeWarning: invalid value encountered in matmul
  return D_inv_sqr @ S @ D_inv_sqr



1: 
2: 
3: 
4: 
5: 
6: 
7: 
8: 
9: 
10: 
11: 
12: 
13: 
14: 
15: 
16: 
17: 
18: 
19: 
20: 
21: 
22: 
23: 
24: 

WisconsinDataset
0: 
1: 
2: 
3: 
4: 
5: 
6: 
7: 
8: 
9: 
10: 
11: 
12: 
13: 
14: 
15: 
16: 
17: 
18: 
19: 
20: 
21: 
22: 
23: 
24: 

CornellDataset
0: 
1: 
2: 
3: 
4: 
5: 
6: 
7: 
8: 
9: 
10: 
11: 
12: 
13: 
14: 
15: 
16: 
17: 
18: 
19: 
20: 
21: 
22: 
23: 
24: 

CiteseerGraphDataset
0: 
1: 
2: 
3: 
4: 
5: 
6: 
7: 
8: 
9: 
10: 
11: 
12: 
13: 
14: 
15: 
16: 
17: 
18: 
19: 
20: 
21: 
22: 
23: 
24: 

CoraGraphDataset
0: 
1: 
2: 
3: 
4: 
5: 
6: 
7: 
8: 
9: 
10: 
11: 
12: 
13: 
14: 
15: 
16: 
17: 
18: 
19: 
20: 
21: 
22: 
23: 
24: 

TexasDataset (homophily ratio: 0.108)
	- Kipf-normA:	mean: 0.583 - std: 0.0562 - med: 0.568 - time: 0.04 mins
	- Kipf-A:	mean: 0.676 - std: 0.0405 - med: 0.676 - time: 0.02 mins
	- GCN-normA:	mean: 0.108 - std: 0.0000 - med: 0.108 - time: 0.02 mins
	- GCN-A:	mean: 0.617 - std: 0.0699 - med: 0.622 - time: 0.02 mins
	- GCN-normA-1:	mean: 0.108 - std: 0.

In [6]:
table_acc_val1

,TexasDataset (0.11),WisconsinDataset (0.20),CornellDataset (0.13),CiteseerGraphDataset (0.74),CoraGraphDataset (0.81)
Kipf-normA,0.582703,0.552157,0.418378,0.70200,0.80848
Kipf-A,0.675676,0.545882,0.401081,0.64780,0.75296
GCN-normA,0.108108,0.019608,0.270270,0.68904,0.79700
GCN-A,0.617297,0.516863,0.503784,0.61552,0.71724
GCN-normA-1,0.108108,0.019608,0.270270,0.68348,0.77128
GCN-A-1,0.467027,0.407843,0.370811,0.62692,0.71024
GCN-A-4-2,0.569730,0.447843,0.512432,0.43896,0.55780
GCN-normH,0.507027,0.550588,0.440000,0.67524,0.77144
GCN-H,0.461622,0.414118,0.381622,0.62252,0.71536
W-GCN-normA,0.108108,0.019608,0.270270,0.69116,0.77668


In [7]:
table_acc1

,TexasDataset (0.11),WisconsinDataset (0.20),CornellDataset (0.13),CiteseerGraphDataset (0.74),CoraGraphDataset (0.81)
Kipf-normA,0.665946,0.591373,0.539459,0.71060,0.81396
Kipf-A,0.717838,0.578039,0.446486,0.65436,0.76372
GCN-normA,0.108108,0.019608,0.270270,0.69856,0.80584
GCN-A,0.696216,0.559216,0.574054,0.62272,0.72216
GCN-normA-1,0.108108,0.019608,0.270270,0.68836,0.77620
GCN-A-1,0.548108,0.477647,0.435676,0.63680,0.71756
GCN-A-4-2,0.654054,0.487843,0.580541,0.44868,0.56544
GCN-normH,0.610811,0.587451,0.499459,0.68404,0.77516
GCN-H,0.548108,0.476078,0.450811,0.63132,0.72000
W-GCN-normA,0.108108,0.019608,0.270270,0.69704,0.78272


## Best Params GF-GNN

In [8]:
# BEST PARAMETERS
## Reaining params
N_RUNS = 25
N_EPOCHS = 200  # 500
LR = .05
WD = .01
DROPOUT = 0

# BEST PARAMETERS
## Architecture params
N_LAYERS = 2
K = 3
HID_DIM = 50

## Model params
h0 = 1
ACT = nn.ReLU()
LAST_ACT = nn.Softmax(dim=1)
LOSS_FN = nn.CrossEntropyLoss()

EXPS = [{'model': 'Kipf', 'norm': 'both', 'leg': 'Kipf-normA'},
        {'model': 'Kipf', 'norm': 'none', 'leg': 'Kipf-A'},
        {'model': 'GFGCN', 'L': N_LAYERS, 'K': K, 'h0': h0, 'norm': True, 'leg': 'A-GCN-normA'},
        {'model': 'GFGCN', 'L': N_LAYERS, 'K': K, 'h0': h0, 'norm': False, 'leg': 'A-GCN'},
        {'model': 'H-GFGCN', 'L': N_LAYERS, 'K': K, 'norm': True, 'leg': 'H-GCN-normH'},
        {'model': 'H-GFGCN', 'L': N_LAYERS, 'K': K, 'norm': False, 'leg': 'GCN-H'},
        {'model': 'noh-GFGCN', 'L': N_LAYERS, 'K': K, 'norm': True, 'leg': 'W-GCN-normA'},
        {'model': 'noh-GFGCN', 'L': N_LAYERS, 'K': K, 'norm': False, 'leg': 'W-GCN'},]

In [9]:
best_accs = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
accs_best_val = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
ellapsed_times = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
for j, dataset in enumerate(DATASETS):
    A, feat, labels, n_class, masks = utils.get_data_dgl(dataset, dev=device)
    N = A.shape[0]
    in_dim = feat.shape[1]
    out_dim = n_class

    print(dataset)

    for i in range(N_RUNS):
        print(f'{i}:', end=' ')
        for k, exp in enumerate(EXPS):
            t_i = time.time()
            if exp['model'] == 'Kipf':
                arch = GCNN_2L(in_dim, HID_DIM, out_dim, act=ACT, last_act=LAST_ACT,
                               dropout=DROPOUT, norm=exp['norm'])
                S = dgl.from_networkx(nx.from_numpy_array(A)).add_self_loop().to(device)

            elif exp['model'] == 'GFGCN':
                arch = GFGCN(in_dim, HID_DIM, out_dim, exp['L'], exp['K'], act=ACT, last_act=LAST_ACT,
                         dropout=DROPOUT, diff_layer=GFGCNLayer, init_h0=exp['h0'])
                if exp['norm']:
                    S = torch.Tensor(normalize_gso(A, 'both')).to(device)
                else:
                    S = torch.Tensor(A).to(device)

            elif exp['model'] == 'H-GFGCN':
                arch = GFGCN_Spows(in_dim, HID_DIM, out_dim, exp['L'], exp['K'], act=ACT, last_act=LAST_ACT,
                                   dropout=DROPOUT, norm=exp['norm'], dev=device)
                S = torch.Tensor(A).to(device)

            elif exp['model'] == 'noh-GFGCN':
                arch = GFGCN(in_dim, HID_DIM, out_dim, exp['L'], exp['K'], act=ACT, last_act=LAST_ACT,
                             dropout=DROPOUT, diff_layer=GFGCN_noh_Layer)
                if exp['norm']:
                    S = torch.Tensor(normalize_gso(A, 'both')).to(device)
                else:
                    S = torch.Tensor(A).to(device)
            else:
                raise Exception(f'ERROR: unknown architecture {exp["model"]}')

            if exp['model'] in ['Kipf', 'noh-GFGCN']:
                model = NodeClassModel(arch, S, masks, LOSS_FN, device=device)
            else:
                model = GF_NodeClassModel(arch, S, exp['K'], masks, LOSS_FN, device=device)

            loss, acc = model.train(feat, labels, N_EPOCHS, LR, WD)
            ellapsed_t = (time.time()-t_i)/60
            
            
            best_accs[k,j,i] = np.max(acc["test"])
            accs_best_val[k,j,i] = acc["test"][np.argmax(acc["val"])]

            ellapsed_times[k,j,i] = ellapsed_t

            print(f'{accs_best_val[k,j,i]:.3f} ({best_accs[k,j,i]:.3f})', end=' - ')
        print()      
    print()


print_full_results(accs_best_val, ellapsed_times, DATASETS, EXPS)
table_acc2 = summary_table(best_accs, DATASETS, EXPS)
table_acc_val2 = summary_table(accs_best_val, DATASETS, EXPS)

TexasDataset
0: 0.595 (0.703) - 0.703 (0.730) - 

/home/srey/Investigacion/robust_minmax_gnn/gsp_utils/data.py:43: RuntimeWarning: divide by zero encountered in divide
  D_inv = np.diag(1/S.sum(1))
/home/srey/Investigacion/robust_minmax_gnn/gsp_utils/data.py:51: RuntimeWarning: invalid value encountered in matmul
  return D_inv_sqr @ S @ D_inv_sqr


0.108 (0.108) - 0.811 (0.919) - 0.838 (0.838) - 0.811 (0.919) - 0.108 (0.108) - 0.757 (0.784) - 
1: 0.676 (0.703) - 0.649 (0.649) - 0.108 (0.108) - 0.730 (0.919) - 0.703 (0.757) - 0.730 (0.865) - 0.108 (0.108) - 0.730 (0.757) - 
2: 0.703 (0.703) - 0.730 (0.730) - 0.108 (0.108) - 0.865 (0.892) - 0.541 (0.676) - 0.838 (0.892) - 0.108 (0.108) - 0.730 (0.757) - 
3: 0.676 (0.730) - 0.703 (0.703) - 0.108 (0.108) - 0.811 (0.865) - 0.676 (0.838) - 0.811 (0.865) - 0.108 (0.108) - 0.676 (0.730) - 
4: 0.676 (0.703) - 0.703 (0.703) - 0.108 (0.108) - 0.865 (0.892) - 0.649 (0.703) - 0.811 (0.892) - 0.108 (0.108) - 0.676 (0.757) - 
5: 0.649 (0.676) - 0.459 (0.649) - 0.108 (0.108) - 0.811 (0.892) - 0.784 (0.811) - 0.865 (0.892) - 0.108 (0.108) - 0.730 (0.757) - 
6: 0.622 (0.703) - 0.649 (0.649) - 0.108 (0.108) - 0.811 (0.865) - 0.757 (0.757) - 0.865 (0.892) - 0.108 (0.108) - 0.757 (0.784) - 
7: 0.730 (0.730) - 0.676 (0.676) - 0.108 (0.108) - 0.838 (0.892) - 0.730 (0.811) - 0.865 (0.865) - 0.108 (0.108

In [10]:
table_acc_val2

,TexasDataset (0.11),WisconsinDataset (0.20),CornellDataset (0.13),CiteseerGraphDataset (0.74),CoraGraphDataset (0.81)
Kipf-normA,0.664865,0.616471,0.532973,0.19180,0.31736
Kipf-A,0.671351,0.526275,0.410811,0.60296,0.75112
A-GCN-normA,0.108108,0.019608,0.270270,0.71488,0.80976
A-GCN,0.822703,0.796078,0.774054,0.57584,0.58268
H-GCN-normH,0.681081,0.651765,0.660541,0.20788,0.32552
GCN-H,0.824865,0.803922,0.763243,0.54232,0.56236
W-GCN-normA,0.108108,0.019608,0.270270,0.22468,0.31636
W-GCN,0.702703,0.607059,0.570811,0.52020,0.42116


In [11]:
table_acc2

,TexasDataset (0.11),WisconsinDataset (0.20),CornellDataset (0.13),CiteseerGraphDataset (0.74),CoraGraphDataset (0.81)
Kipf-normA,0.710270,0.672157,0.625946,0.23556,0.31964
Kipf-A,0.682162,0.545882,0.434595,0.60612,0.76092
A-GCN-normA,0.108108,0.019608,0.270270,0.72704,0.81652
A-GCN,0.886486,0.854902,0.860541,0.58256,0.58820
H-GCN-normH,0.763243,0.729412,0.735135,0.24324,0.32596
GCN-H,0.887568,0.858039,0.849730,0.55024,0.56584
W-GCN-normA,0.108108,0.019608,0.270270,0.25728,0.31800
W-GCN,0.757838,0.683137,0.616216,0.52556,0.42324
